In [ ]:
# common
import os
BASE_DIR = os.path.dirname(os.path.realpath('__file__'))

from datetime import datetime
from io import BytesIO
from pathlib import Path

# network & protocol
import requests
from requests.exceptions import BaseHTTPError

# computational
import numpy as np
import pandas as pd

In [ ]:
start_date = '2013-01-01'
end_date = '2020-12-31'
date_tail = 'T00:00:00.000Z'
stations = [
    {
        'name': 'Neak_Luong',
        'stage_id': '0c55bc1fb44e4caf881226bbf8a6e62a',
    },
    {
        'name': 'Stung_Treng',
        'discharge_id': '2066a06f57764afca57a6be0f3eff8a7',
    },
    {
        'name': 'Chaktomuk',
        'stage_id': '7f84c4d851a543d1a5eccc53947e0d3c',
    },
    {
        'name': 'Kompong_Kdei',
        'stage_id': '75ba98758e7c4eb3b105f733db150307',
    },
    {
        'name': 'Prek_Kdam',
        'discharge_id': '6615982ab17b4a38a5269e5ba4ef8a4f',
        'stage_id': '81e259eb15f64135834fe9e388ee7fbb',
    },
    {
        'name': 'Kg._Thmar',
        'discharge_id': '2575ba232eb1406a8ce460de9fd5849a',
        'stage_id': '549c7f63d6dc4392b233b2049bd23d11',
        'start_date': '1962-03-31',
        'end_date': '2002-12-30',
    },
    {
        'name': 'Koh_Khel',
        'discharge_id': 'e2b360019f3f4c438766ab8ed04286aa',
        'stage_id': '64758ae8f24a46759d962ab9a2dea27a',
    },
    {
        'name': 'Lumphat',
        'discharge_id': 'f17adb5f893a4f40924f698978906b87',
        'stage_id': '93337ffe7838467289bf5b55401ee6f8',
    },
    {
        'name': 'Kompong_Cham',
        'discharge_id': '30f15c16035f4bcfbac36fc545fb132a',
        'stage_id': 'b5da800e9c8e4346a3b8d9d6daa73e0a',
    },
    {
        'name': 'Phnom_Penh_Port',
        'discharge_id': 'e63fa05dafea4873bd44574125a97495',
        'stage_id': 'cd94408bcf944bc88033cf05675034fd',
    },
    {
        'name': 'Voeun_Sai',
        'discharge_id': 'eacd08c3a27e4fc5a3aa1af35792f6e3',
        'stage_id': 'd177b0f2dae547ceb9bc7c23ef2970ae',
    },
    {
        'name': 'Kratie',
        'discharge_id': '17c0ffe74e224b50a9e3142fc404262b',
        'stage_id': 'fca7bee14dcf43a59263158f18a88509',
    },
]